In [1]:
import somhos.resources.dataset as rd
import somhos.resources.queries as rq
import kleis.resources.dataset as kl
import hashlib as hl

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Default:  ~/kleis_data/corpus/semeval2017-task10


    - Download from here https://scienceie.github.io/resources.html
    - Use one of the following paths.
        + ./kleis_data/corpus/semeval2017-task10/
        + ~/kleis_data/corpus/semeval2017-task10/
        + /home/jupyterlab/.local/lib/python3.5/site-packages/kleis/kleis_data/corpus/semeval2017-task10/
    - You can use pre-trained models.


In [2]:
kleis = kl.load_corpus()
kleis.training(features_method="simple-posseq", filter_min_count=10)

### [Aminer Citation Network Datasets - v1 and v9](https://aminer.org/citation)



In [3]:
#data_path = "resources/aminer/v1"
data_path = "../../src/somhos/resources/aminer/v9"
keyphrases_path = "../../src/somhos/resources/aminer/v9/kp-simple-posseq-10-nopostprocessing.txt"

In [4]:
related_docs = True
# Index 
ix_data = rq.cur_indexed_docs(data_path)
print(ix_data)

FileIndex(FileStorage('../../src/somhos/resources/aminer/v9/index-data'), 'MAIN')


In [ ]:
with ix_data.reader() as reader:
    i = 0
    kps_count = 0
    kps_count_set = 0
    kps_count_hashes = 0
    kps_hashes_9 = {}
    kps_hashes = {}
    for doc_id, doc in reader.iter_docs():
        i += 1
        if i < 900000:
            continue
        if i >= 1000000:
            break
        try:
            kps = set()
            keyphrases = kleis.label_text(doc['content'], post_processing=False)
            kps_count += len(keyphrases)
            for k in keyphrases:
                keyphrase = k[2].lower().encode('utf-8')
                kps.add(keyphrase)
                kps_hash = hl.md5(keyphrase).hexdigest()
                kps_hashes_9.setdefault(kps_hash[:9], set())
                kps_hashes.setdefault(kps_hash, set())
                # kps_hashes[kps_hash].add(keyphrase)
            kps_count_set += len(kps)
        except ValueError:
            pass
    kps_count_hashes = len(kps_hashes)
    kps_count_hashes_9 = len(kps_hashes_9)
        # if len(kps) > 2:
    # print(kps_hashes)
    print(" - Documents: %d" % (i))
    print(" - Keyphrases: %d, %d, %d, %d" % (kps_count, kps_count_set, kps_count_hashes, kps_count_hashes_9))